# Week 1 worksheet

In this worksheet we will explore some of the basic concepts of statistical pattern analysis and linear data projection.

## Gaussian distribution


In [1]:
import matplotlib.pyplot as plt
import numpy as np

We start by generating 3000 data values from a one-dimensional Gaussian with mean 2 and variance 5. Note how the seed of the random number generator has been fixed to enable us to reproduce results reliably. This is crucial for debugging and auditing applications of data science.

In [2]:
from numpy.random import Generator, PCG64
mean = 2
sigma = np.sqrt(5)
rng = Generator(PCG64(12345))
vals = mean + sigma*rng.standard_normal(3000)

Plot a histogram of this data with 30 bins

In [3]:
#TODO

Now compute the maximum likelihood estimates of the mean and standard deviation of the data. Print the values and make a note of them.

In [4]:
#TODO

That is one result for maximum likelihood estimation based on a single sample from the distribution. Now we will do this 100 times with a smaller sample (of size 10) and compare the average estimate of the mean (over the 100 iterations) with its true value and the average estimate of the variance (over the 100 iterations) with its true value. (Continue with the random number generator in its current state (i.e. do not reset it)). Print out these values and make a note of them for the quiz later.

In [5]:
#TODO

## Principal Component Analysis (PCA) for data projection

In this section we will explore the theory and practice of PCA for data projection.

We start by reading in the oil flow dataset mentioned in the lectures. The ID field is simply the index of the row in the dataset (starting from 1). The variables have the imaginative names v1, v2, etc. While the final column is the class label, which will be treated separately.

In [6]:
import pandas 

df = pandas.read_csv('oil.csv', index_col='ID')
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,Label
ID,,,,,,,,,,,,,
1,0.3315,0.2156,0.6802,0.1434,0.6825,0.2720,0.6223,0.2092,0.7961,0.1530,0.5856,0.2573,1
2,0.5184,0.2283,0.5300,0.6884,0.7456,0.6171,0.6136,0.5928,0.7678,0.6130,0.6705,0.5202,1
3,0.0760,0.5010,0.1870,0.7011,0.1728,0.8475,0.2300,0.6536,0.1616,0.8732,0.1603,0.7331,1
4,0.3334,0.5468,0.6072,0.7549,0.6294,0.9322,0.5850,0.7859,0.6324,0.9568,0.6026,0.7616,1
5,0.1703,0.2234,0.2677,0.3477,0.2734,0.4324,0.1222,0.5213,0.2333,0.4926,0.2044,0.4215,1


Next, compute the covariance matrix of the data (just v1 to v12) and then carry out an eigendecomposition: check the `numpy` definitions of the appropriate functions. Store the eigenvalues in a variable `eval` and the eigenvectors in a variable `evec`. Start by using the `eig` function. 

Note that in the `eig` function each row of the matrix parameter represents a variable, and each column a single observation of all those variables. This is the opposite to the convention in Matlab and in Chris Bishop's book. Either apply a transpose operator or set the `rowvar` parameter to `false`.

In [7]:
#TODO



Now we list and plot the eigenvalues. 

In [8]:
#Uncomment code when previous section is completed
#print (eval)
#plt.figure()
#plt.subplot(211)
#plt.plot( eval, 'ok')


We can see the effect of different algorithms for calculating the eigenvalues and eigenvectors. The `eigh` function is specialised for working with symmetric matrices: it is faster and more accurate than `eig` in this case. Use it to compute eigenvalues and eigenvectors for this task (again with the same variable names). 

In [9]:
#TODO - then uncomment plotting code

#plt.subplot(212)
#plt.plot( eval, 'ok')
#plt.show()

The next task is to sort the eigenvalues in descending order (If we were using the vectors to project the data, we would have to rearrange the eigenvectors correspondingly). 

Then you should write code to compute the cumulative sums of the resulting vector and express the values as a fraction of the total of the eigenvalues (compare with the lecture notes). Plot the cumulative sum. I have provided code to draw a threshold at 0.95.

In [10]:
# Sort the eigenvalues - should also reorder the eigenvectors if applying them
# Uncomment code when TODO section is complete
#sorted_eval = np.sort(eval)
#eval = sorted_eval[::-1]  # reverse the order so that it is decreasing
#print("Sorted eigenvalues " + str(eval))

# TODO Compute cumulative sum


# Plot cumulative sum of eigenvalues with a threshold at 0.95
# Uncomment code when TODO section is complete
#plt.figure(3)
#plt.plot(cumuleval, 'ok')
#xvals = range(ncols-1)
#plt.plot(xvals, 0.95*np.ones(np.shape(eval)), 'r-')
#plt.show()

Now write code to apply PCA (look up the function definition) with 2 principal components and then project the data onto this space using the `transform` function (which automatically subtracts off the mean of each variable). Call the resulting variable `projX`. Generate a scatter plot of the result.

In [11]:
# Now apply PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)
# This should equal the sum of the first two values in the cumulative proportion plot
print('Explained variance ratio '+ str(pca.explained_variance_ratio_))

# Project data onto first two principal components
projX = pca.transform(X)
plt.figure(4)
plt.plot(projX[:,0], projX[:,1], 'k.')
plt.title('Projected data')
plt.show()

NameError: name 'X' is not defined

This graph is interesting - it shows the structure of the data quite well in terms of clusters and outliers. But we also have extra information that we can plot: namely the class labels. The following piece of code uses the label to colour each mark: red circle, green cross, and blue square for labels 1, 2, and 3 respectively.

In [ ]:
# Now plot the data with a different mark for each label
marks = ('ro', 'g+', 'bs')
plt.figure(6)
for n in range(1, 4, 1) :
    projClass = projX[np.equal(label, n*np.ones(label.shape))]
    plt.plot(projClass[:,0], projClass[:,1], marks[n-1])

plt.title('Projected data showing label')
plt.show()

## PCA: dimensionality reduction

In this section, we will explore the use of PCA for dimensionality reduction and reconstruction. We will work with the digit dataset provided by `sklearn`, though it does have drawbacks given the small $8 \times 8$ size of the images.

Here we load the data and then display the first image, which is a zero.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
 
plt.gray() 
plt.matshow(digits.images[0]) 
plt.show() 

If you haven't seen this dataset before, it is useful to know what the objects are stored with it. Note that `data` is a matrix with one digit per row and 64 columns (representing the 64 pixel values) while `images` is a list of images each of which is an $8\times 8$ matrix.

In [ ]:
print(digits.keys())

Next, compute the top 30 principal components of the `data` and then show the first component as an image. N.B. You will need to reshape the component as an $8\times 8$ matrix before plotting it.

In [ ]:
#TODO

Next you should write code to project the first few rows (say $10$) of the data to the PCA subspace (using `transform`) and then _reconstruct_ the first digit in the original space using the `inverse_transform` function. Display the reconstructed digit as an image as before.

In [ ]:
#TODO

By printing the first few values of the targets, we see that the digits are encoded by the corresponding number 0-9. 

In [ ]:
print(digits.target[0:20])


You can use this fact to create a subset of the data consisting only of the digit 3. Write code to do this (you may find it useful to look at the indexing trick in the definition of `projClass` above) and then compute and show the first principal component. We can call it an _eigendigit_. How does this compare with the principal component computing on the entire dataset? Why do you think there is a difference?

In [ ]:
#TODO

The eigendigit for the entire dataset shows all the variations in all the digits so is not recognisably related to any specific digit. The low resolution makes interpretation difficult, but there is some indication that the eigendigit for 3 does have some structure in it.

## Fisher linear discriminant
In this final exercise, we will explore the Fisher linear discriminant implementation and compare the results with PCA. We will work with the oil data again and extract the last column to a vector `label`.

In [ ]:
# Work with oil data again

df = pandas.read_csv('oil.csv', index_col='ID')
df.head()

data = df.to_numpy()
ncols = data.shape[1]

X = data[:,0:ncols-1]
label = data[:,ncols-1]

Now apply `LinearDiscriminantAnalysis` from `sklearn` and project the data to a variable `projX`. The rest of the code in the next block plots the data in the same way as for PCA above. Write down two differences between the PCA and Fisher discriminant projections.

In [ ]:
#TODO

# Now plot the data with a different mark for each label
marks = ('ro', 'g+', 'bs')
plt.figure()
for n in range(1, 4, 1) :
    projClass = projX[np.equal(label, n*np.ones(label.shape))]
    plt.plot(projClass[:,0], projClass[:,1], marks[n-1])

plt.title('Projected data showing label')
plt.show()